In [1]:
import requests
import json
import pandas as pd
from sqlalchemy import create_engine, types
from dotenv import load_dotenv
import numpy as np
import math 

load_dotenv()

import os

In [2]:
def generate_schema(df):

    schema = {}

    for c in df.columns:

        if df[c].dtype == np.dtype("int64"):
            schema[c] = types.BIGINT()
        elif df[c].dtype == np.dtype("float64"):
            schema[c] = types.Float(asdecimal=True)
        elif df[c].dtype == np.dtype("datetime64[ns]"):
            schema[c] = types.DateTime(timezone=False)
        elif df[c].dtype == np.dtype("bool"):
            schema[c] = types.BOOLEAN()
        else:
            schema[c] = types.VARCHAR(length=65535)

    return schema

<b>[Parcel data](https://www.opendataphilly.org/dataset/property-parcels)</b> and <b>[property assessment data](https://www.opendataphilly.org/dataset/opa-property-assessments)</b> links on Philly's Open Data site. <b>[Business (rental) license data](https://www.opendataphilly.org/dataset/licenses-and-inspections-business-licenses/resource/bc00c2db-2d8f-421f-94f4-6e24531d661a)</b> can be found on the site too

In [3]:
with open('opa_properties_public.geojson','r') as f:
    properties = json.load(f)

In [4]:
properties_df = pd.json_normalize(properties['features'])
properties_df.head()

,type,geometry.type,geometry.coordinates,properties.assessment_date,properties.basements,properties.beginning_point,properties.book_and_page,properties.building_code,properties.building_code_description,properties.category_code,...,properties.unfinished,properties.unit,properties.utility,properties.view_type,properties.year_built,properties.year_built_estimate,properties.zip_code,properties.zoning,properties.objectid,geometry
0,Feature,Point,"[-75.146866, 39.931278]",None,D,S W COR HOWARD,0000000,P50,ROW W/GAR 3 STY MASONRY,1,...,None,None,None,I,1920,Y,191475425,RSA5,832274881,NaN
1,Feature,Point,"[-75.146921, 39.931286]",None,D,15D94 W HOWARD ST,3209422,O50,ROW 3 STY MASONRY,1,...,None,None,None,I,1920,Y,191475425,RSA5,832274882,NaN
2,Feature,Point,"[-75.146971, 39.931292]",None,D,"30'51/2"" W HOWARD",0000000,O50,ROW 3 STY MASONRY,1,...,None,None,None,I,1920,Y,191475425,RSA5,832274883,NaN
3,Feature,Point,"[-75.147034, 39.93123]",None,D,"45'2"" W HOWARD ST",2448875,O50,ROW 3 STY MASONRY,1,...,None,None,None,I,1920,Y,191475425,RSA5,832274884,NaN
4,Feature,Point,"[-75.147087, 39.931236]",None,C,"59'6"" W HOWARD ST",3245897,O50,ROW 3 STY MASONRY,1,...,None,None,None,I,1920,Y,191475425,RSA5,832274885,NaN


In [5]:
properties_cols = [c.replace('.','_') for c in properties_df.columns]
properties_df.columns = properties_cols

In [6]:
with open('business_licenses.geojson','r') as f:
    licenses = json.load(f)

In [7]:
licenses_df = pd.json_normalize(licenses['features'])
licenses_df.head()

,type,geometry,properties.objectid,properties.addressobjectid,properties.address,properties.unit_type,properties.unit_num,properties.zip,properties.censustract,properties.parcel_id_num,...,properties.legalfirstname,properties.legallastname,properties.legalname,properties.legalentitytype,properties.business_name,properties.business_mailing_address,properties.geocode_x,properties.geocode_y,geometry.type,geometry.coordinates
0,Feature,NaN,1,15698520,None,None,None,None,None,None,...,None,None,509 VINE STREET TCE LP,Company,509 VINE STREET TCE LP,"4328 Ridge Ave PHILADELPHIA, PA 19129 USA",NaN,NaN,NaN,NaN
1,Feature,NaN,2,15499964,4442 SILVERWOOD ST,None,None,19127-1329,214,385822,...,None,None,SOROKIN FRAN,Company,SOROKIN FRAN,"4361 MAIN ST PHILADELPHIA, PA 19127 USA",2.676135e+06,263258.845701,Point,"[-75.222782, 40.028207]"
2,Feature,NaN,3,15793318,198 W ASHDALE ST,None,None,19120-3428,2.86,181798,...,ROM,LAY,LAY ROM,Individual,LAY ROM,"198 W ASHDALE ST PHILADELPHIA, PA 19120 USA",2.702920e+06,263553.000000,Point,"[-75.127149, 40.026871]"
3,Feature,NaN,4,15419931,2922 BOUDINOT ST,None,None,19134-2905,177.02,62823,...,& CHRISTENSEN GARY,PIETRZYKOWSKI STEPHEN,PIETRZYKOWSKI STEPHEN & CHRISTENSEN GARY,Individual,T/A HELP-U-BUY PROPERTIES,"411 DOYLESTOWN RD. NO. 881 MONTGOMERYVILLE, PA...",2.704326e+06,251449.698116,Point,"[-75.123414, 39.993546]"
4,Feature,NaN,5,15531875,8223 GERMANTOWN AVE,None,None,19118-3401,2.27,489492,...,RONALD & BEVERLY,LOUX,LOUX RONALD & BEVERLY,Individual,POPPY'S SEED,"144 COWPATH RD SOUDERTON, PA 18964 USA",2.681330e+06,280186.000000,Point,"[-75.202497, 40.074245]"


In [8]:
licenses_cols = [c.replace('.','_') for c in licenses_df.columns]
licenses_df.columns = licenses_cols

In [9]:
licenses_df.head()

,type,geometry,properties_objectid,properties_addressobjectid,properties_address,properties_unit_type,properties_unit_num,properties_zip,properties_censustract,properties_parcel_id_num,...,properties_legalfirstname,properties_legallastname,properties_legalname,properties_legalentitytype,properties_business_name,properties_business_mailing_address,properties_geocode_x,properties_geocode_y,geometry_type,geometry_coordinates
0,Feature,NaN,1,15698520,None,None,None,None,None,None,...,None,None,509 VINE STREET TCE LP,Company,509 VINE STREET TCE LP,"4328 Ridge Ave PHILADELPHIA, PA 19129 USA",NaN,NaN,NaN,NaN
1,Feature,NaN,2,15499964,4442 SILVERWOOD ST,None,None,19127-1329,214,385822,...,None,None,SOROKIN FRAN,Company,SOROKIN FRAN,"4361 MAIN ST PHILADELPHIA, PA 19127 USA",2.676135e+06,263258.845701,Point,"[-75.222782, 40.028207]"
2,Feature,NaN,3,15793318,198 W ASHDALE ST,None,None,19120-3428,2.86,181798,...,ROM,LAY,LAY ROM,Individual,LAY ROM,"198 W ASHDALE ST PHILADELPHIA, PA 19120 USA",2.702920e+06,263553.000000,Point,"[-75.127149, 40.026871]"
3,Feature,NaN,4,15419931,2922 BOUDINOT ST,None,None,19134-2905,177.02,62823,...,& CHRISTENSEN GARY,PIETRZYKOWSKI STEPHEN,PIETRZYKOWSKI STEPHEN & CHRISTENSEN GARY,Individual,T/A HELP-U-BUY PROPERTIES,"411 DOYLESTOWN RD. NO. 881 MONTGOMERYVILLE, PA...",2.704326e+06,251449.698116,Point,"[-75.123414, 39.993546]"
4,Feature,NaN,5,15531875,8223 GERMANTOWN AVE,None,None,19118-3401,2.27,489492,...,RONALD & BEVERLY,LOUX,LOUX RONALD & BEVERLY,Individual,POPPY'S SEED,"144 COWPATH RD SOUDERTON, PA 18964 USA",2.681330e+06,280186.000000,Point,"[-75.202497, 40.074245]"


In [10]:
def get_lat(x):
    
    if isinstance(x,list):
        if len(x) == 2: 
            return x[1]
        else:
            return np.nan
    else:
        return np.nan

def get_lon(x):
    
    if isinstance(x,list):
        if len(x) == 2: 
            return x[0]
        else:
            return np.nan
    else:
        return np.nan
        

In [11]:
licenses_df['latitude'] = licenses_df.geometry_coordinates.apply(lambda x: get_lat(x))
properties_df['latitude'] = properties_df.geometry_coordinates.apply(lambda x:  get_lat(x))

licenses_df['longitude'] = licenses_df.geometry_coordinates.apply(lambda x: get_lon(x))
properties_df['longitude'] = properties_df.geometry_coordinates.apply(lambda x: get_lon(x))

In [12]:
db_url = os.getenv('DB_URL')
eng = create_engine(db_url)

In [13]:
properties_drop_cols = [
    'geometry',
    'type',
    'geometry_type',
    'geometry_coordinates'
    
]
properties_df = properties_df.drop(columns=properties_drop_cols)

In [14]:
licenses_drop_cols = [
    'geometry',
    'type',
    'geometry_type',
    'geometry_coordinates'
    
]
licenses_df = licenses_df.drop(columns=licenses_drop_cols)

In [15]:
licenses_df.to_sql('licenses',
                   eng,if_exists='replace',
                   index=False,
                   chunksize=10000,
                   method='multi',
                   dtype=generate_schema(licenses_df)
                  )

In [16]:
properties_df.to_sql('properties',
                     eng,if_exists='replace',
                     index=False,chunksize=10000,
                     method='multi',
                     dtype=generate_schema(properties_df)
                    )